# Project: Machine Learning Capstone - Predict Movie Revenue

This project works with the datasets from the Kaggle competition at https://www.kaggle.com/c/tmdb-box-office-prediction/data to create a model for predicting worldwide box office revenue for movies.

There are 3000 movies in the sample data set being used for training and testing. The model created from this will then be used on a separate dataset to score the accuracy of predictions. The sample set contains 23 columns, of which 7 columns each contain JSON array data of objects. These objects, along with other fields, will require pre-processing into individual table columns for use in modeling. The fields the will be focuse on are:

* **belongs_to_collection**: Indicates whether this movie is part of a series, and if so which series. Will be one-hot encoded as col_0 for none or col_X where X is the associated collection.
* **budget**: Budget for film as an integer. Some entries have a budget of 0. It will be interesting to try the model both with and without this column as a consideration.
* **genres**: Indicates the genres to which the film belongs. Will be one-hot encoded as gen_X where X is the associated genre.
* **homepage**: Lists the URL for the homepage of the movie, if any. Will be encoded to 0 or 1 to indicate only whether the movie had a homepage.
* **original_language**: Gives the ISO language value for the film. Will be one-hot encoded to lan_ISO where ISO is the ISO language value.
* **popularity**: Long number value rating the film's popularity. This would not be a known value for future film productions, so it will be interesting to try modeling both with and without this value considered.
* **production_companies**: Indicates production companies involved in the movie. Will be one-hot encoded as pcom_X where X is the id of the production company.
* **production_countries**: Indicates countries where the movie was filmed or produced. Will be one-hot encoded as pcou_ISO where ISO is the ISO_3166_1 country value.
* **release_date**: The release date for the film in mm/dd/yyyy format. Will be encoded to r_year and r_week columns, where r_week is the number of the week in the year in which the file was relesed (0-51)
* **runtime**: Integer value for the runtime of the film in minutes.
* **spoken_languages**: Indicates the languages spoken in the movie. Will be one-hot encoded to spo_ISO where ISO is the ISO_639-1 language value.
* **Keywords**: Indicates the keyword values associated with the movie. Keyword objects have the format {'id': int, 'name': ''}, so these will be encoded to key_X where X is the keyword id.
* **cast**: Indicates cast members associated with the movie. Will be one-hot encoded to cast_X where X is the id of the cast member.
* **crew**: Indicates crew members associated with the movie. Will be one-hot encoded to crew_X where X is the id of the crew member.
* **revenue**: Integer value for the worldwide revenue of the movie.

In [1]:
# Import libraries necessary for this project
import sys
sys.path.insert(0, 'utilities')

import numpy as np
import pandas as pd
import json
import ast
from pandas.io.json import json_normalize
from sklearn.model_selection import ShuffleSplit

from json_columnizer import jcolumnize
from json_columnizer import crew_columnize
from json_columnizer import columnBooleanize
from json_columnizer import originalLanguage
from fix_missing_data import fixTrainRevenueAndBudget
from fix_missing_data import fixTrainRuntime
from fix_missing_data import fixLowDollars
from json_columnizer import columnizeDates

# Import supplementary visualizations code visuals.py
#import visuals as vs

# Pretty display for notebooks
%matplotlib inline


# Load the movies dataset
train = pd.read_csv('data/train.csv')

# Drop useless columns
train = train.drop(['imdb_id', 'poster_path', 'original_title', 'overview', 'poster_path',\
                    'status', 'tagline', 'title'], axis = 1)

# Fill missing revenue and budget numbers
train = fixTrainRevenueAndBudget(train)

# Fill missing runtime numbers
train = fixTrainRuntime(train)

# Fill dollar amounts represent in millions
train = fixLowDollars(train)

# Columnize dates
train = columnizeDates(train, 'release_date')
train = train.drop('release_date', axis=1)

# Booleanize homepage column
train = columnBooleanize(train, 'homepage')
train = train.drop('homepage', axis=1)

# Turn original language into columns
train = originalLanguage(train, 'original_language', 'olang_')
train = train.drop('original_language', axis=1)

# Turn crew members into the appropriate columns
# Director
train = crew_columnize(train, 'crew', 'id', 'Director', 'director_')
# Producer
train = crew_columnize(train, 'crew', 'id', 'Producer', 'producer_')
# Executive Producer
train = crew_columnize(train, 'crew', 'id', 'Executive Producer', 'execprod_')
train = train.drop('crew', axis=1)

# Turn collections into booleans
train = columnBooleanize(train, 'belongs_to_collection')
train = train.drop('belongs_to_collection', axis=1)

# Turn cast into columns
train = jcolumnize(train, 'cast', 'id', 'cast_', 4)
train = train.drop('cast', axis=1)

# Turn genres into columns
train = jcolumnize(train, 'genres', 'id', 'genres_')
train = train.drop('genres', axis=1)

# Turn production companies into columns
train = jcolumnize(train, 'production_companies', 'id', 'pcomp_')
train = train.drop('production_companies', axis=1)

# Turn production countries into columns
train = jcolumnize(train, 'production_countries', 'iso_3166_1', 'pcoun_')
train = train.drop('production_countries', axis=1)

# Turn spoken languages into columns
train = jcolumnize(train, 'spoken_languages', 'iso_639_1', 'spoken_')
train = train.drop('spoken_languages', axis=1)

# Turn keywords into columns
train = jcolumnize(train, 'Keywords', 'id', 'key_')
train = train.drop('Keywords', axis=1)

#print(train.head(3))

# Success
print("Movies dataset has {} data points with {} variables each.".format(*train.shape))
#print("Column names {}".format(list(data)))


Columnized original_language shape: (3000, 52)
Columnized crew shape: (3000, 2094)
Exception doing literal eval on crew
Columnized crew shape: (3000, 5648)
Exception doing literal eval on crew
Columnized crew shape: (3000, 7945)
Columnized cast shape: (3000, 13991)
Columnized genres shape: (3000, 14010)
Columnized production_companies shape: (3000, 17721)
Columnized production_countries shape: (3000, 17794)
Columnized spoken_languages shape: (3000, 17872)
Columnized Keywords shape: (3000, 25271)
Movies dataset has 3000 data points with 25270 variables each.


In [2]:
train_revenue = train['revenue']
train_features = train.drop('revenue', axis = 1)

In [3]:
# Explore some basic statistics regarding revenue
train['revenue'] = train_revenue
minimum_rev = np.amin(train_revenue)
maximum_rev = np.amax(train_revenue)
mean_rev = np.mean(train_revenue)
median_rev = np.median(train_revenue)
std_rev = np.std(train_revenue)

print("Stats for dataset:\n")
print("Minimum revenue: ${}".format(minimum_rev))
print("Maximum revenue: ${}".format(maximum_rev))
print("Mean revenue: ${}".format(mean_rev))
print("Median revenue: ${}".format(median_rev))
print("Standard deviation of revenue: ${}".format(std_rev))

Stats for dataset:

Minimum revenue: $1
Maximum revenue: $1519557910
Mean revenue: $66724027.116333336
Median revenue: $16928630.0
Standard deviation of revenue: $137476234.10861793


In [4]:
# Do final quality inspection and cleanup on the data
print("Checking that data has been reformatted")
label_arr = ['id', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'imdb_id', 'original_language',\
            'original_title', 'overview', 'popularity', 'poster_path', 'production_companies',\
            'production_countries', 'release_date', 'runtime', 'spoken_languages', 'status', 'tagline',\
            'title', 'Keywords', 'cast', 'crew', 'revenue']
count = 0
for label in label_arr:
    try:
        print("Found {}: {}".format(label, train_features.loc(0, label)))
        count = count + 1
    except:
        count = count
if count == 0:
    print("All original fields have been properly transformed and disposed")
else:
    print("All original fields have not been properly dealt with. See above.")
        
print ("Checking for bad value types")
count = 0
columns = list(train_features.columns.values)
for x in columns:
    val = train_features.loc[0, x]
    
    if isinstance(val, str) or isinstance(val, list) or isinstance(val, dict) or isinstance(val, tuple):
        print("Maybe bad column {} : {}".format(x, val))
        count = count + 1
        train_features = train_features.drop(x, axis = 1)
    else:
        train_features.loc[train_features[x].isnull(), x] = 0

    
print("There are {} columns with suspect values".format(count))

# Shuffle and split the training set data to prepare for training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_features, train_revenue, test_size=.2, random_state=42)
print("Training and testing split successful")

Checking that data has been reformatted
All original fields have been properly transformed and disposed
Checking for bad value types
There are 0 columns with suspect values
Training and testing split successful


In [ ]:
import plots

plots.VisualizeModelLearning(train_features, train_revenue)
    

/home/doug/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/doug/anaconda3/lib/python3.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


ShuffleSplit sets: ShuffleSplit(3000, n_iter=10, test_size=0.2, random_state=0)
Visualize training set sizes: [   1  301  600  900 1200 1500 1800 2099 2399]
Evaluating depth 1
Results for depth 1: [-0.46658877  0.29033064  0.32928962  0.31362965  0.3097127   0.30456712
  0.29369417  0.29350156  0.29387471]
Evaluating depth 3
Results for depth 3: [-0.46658877  0.41747464  0.43050712  0.45518694  0.44331894  0.46081221
  0.46953972  0.49456477  0.49517761]
Evaluating depth 4
Results for depth 4: [-0.46658877  0.41065712  0.41998532  0.44124461  0.44966434  0.42892392
  0.41620699  0.46149406  0.48544788]
Evaluating depth 5
Results for depth 5: [-0.46658877  0.40616409  0.41658008  0.42323919  0.43480575  0.41236668
  0.40059177  0.4602489   0.44426385]
Evaluating depth 6
Results for depth 6: [-0.46658877  0.37685592  0.42183512  0.39797306  0.41140013  0.42132189
  0.42542183  0.47183684  0.43065621]
Evaluating depth 10


In [ ]:
# Define root mean squared function
def rmsle(y_true, y_predict):
    sum = 0.0
    for x in range(len(y_predict)):
        if y_predict[x] < 0 or y_true.iloc[x] < 0:
            continue
        p = np.log(y_predict[x] + 1)
        r = np.log(y_true.iloc[x] + 1)
        sum = sum + (p - r) ** 2
    return (sum / len(y_predict)) ** 0.5

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

def fit_model(X,y):
    # Create cross-validation sets
    cv_sets = ShuffleSplit(n_splits = 10, test_size = .2, train_size = None, random_state = None)
    # Decision tree regressor
    regressor = DecisionTreeRegressor()
    # Dictionary for max_depth with a range of 1 to 10
    params = {'max_depth': [30, 40, 50, 60, 80, 100], 'min_samples_split': [2, 4, 8, 16, 32, 64, 128], \
              'max_features': [10, 20, 40, 80, 100, 200, 400, 800, 1000]}
    # Use rmsle as the scoring function
    scoring_func = make_scorer(rmsle, greater_is_better=False)
    
    # Create grid search
    grid = GridSearchCV(estimator = regressor, param_grid = params, scoring = scoring_func, cv = cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)
    
    # Return the optimal model after fitting the data
    return grid.best_estimator_


In [ ]:
reg = fit_model(X_train, y_train)

# Check max_depth
print("max_depth is {}".format(reg.get_params()['max_depth']))


In [ ]:
# Make predictions using the model and compare them to the real outcomes
predicted = reg.predict(X_test)
for x in range(20):
    print("Predicted: {}        Real: {}".format(int(predicted[x]), y_test.iloc[x]))

In [ ]:
# Get the score for predicted vs actual
score = rmsle(y_test, predicted)

print("Final model score: {}".format(score))

In [ ]:
# Check max_features and min_samples_split
print("max_features is {}".format(reg.get_params()['max_features']))
print("min_samples_split is {}".format(reg.get_params()['min_samples_split']))